##### Cours : OpenClassRooms

Intervention sur les données avec Pandas (suite)

##### Date : 19-01-23
***

In [1]:
import pandas as pd
import numpy as np

##### <font color = 'green'>Agréger plusieurs lignes (tableaux à une entrée)</font>

On a recours à la méthode .groupby() <br>
Si on a besoin recourir à plusieurs fonctionnalités (somme, moyenne, minimum...),
cette fois-ci, en complément de .groupby(), on a recours à la méthode .agg()

In [2]:
# Importation du fichier 
prets = pd.read_csv('https://raw.githubusercontent.com/benjaminmrl/data-4452741/main/prets.csv')
prets.head()

,identifiant,ville,CP,revenu,remboursement,duree,type,taux
0,0,TOULOUSE,31100,3669.0,1130.05,240,immobilier,1.168
1,1,PARIS,75009,5310.0,240.00,64,automobile,3.701
2,1,PARIS,75009,5310.0,1247.85,300,immobilier,1.173
3,2,MARSEILLE,13010,1873.0,552.54,240,immobilier,0.972
4,3,MARSEILLE,13010,1684.0,586.03,180,immobilier,1.014


In [3]:
# CAHT total pour chacune des agences
prets.groupby('ville').sum()

C:\Users\LRCOM\AppData\Local\Temp\ipykernel_9848\684641079.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  prets.groupby('ville').sum()


,identifiant,CP,revenu,remboursement,duree,taux
ville,,,,,,
BORDEAUX,1718,531800,50816.0,15330.59,3660,18.767
LYON,2833,1932134,95354.0,26064.67,6251,38.918
MARSEILLE,7946,975732,145926.0,40895.72,16420,92.538
NICE,2375,129100,85288.0,23544.24,4460,27.706
PARIS,9849,5776019,335459.0,94052.61,16686,100.073
TOULOUSE,3162,843300,75617.0,19286.18,5643,38.217


In [5]:
# CAHT total pour chacune des agences et par type de prêt
prets.groupby(['ville', 'type']).sum().head()

identifiant       CP    revenu  remboursement  duree  \
ville     type                                                               
BORDEAUX  immobilier         1718   531800   50816.0       15330.59   3660   
LYON      automobile          360   207018   11531.0         788.80    131   
          immobilier         2473  1725116   83823.0       25275.87   6120   
MARSEILLE automobile          404    39031    5842.0         839.60    160   
          immobilier         7542   936701  140084.0       40056.12  16260   

                        taux  
ville     type                
BORDEAUX  immobilier  18.767  
LYON      automobile  10.083  
          immobilier  28.835  
MARSEILLE automobile   9.535  
          immobilier  83.003

In [7]:
# CAHT total pour chacune des agences et par type de prêt 
# mais en affichant en résultat uniquement le champ remboursement
prets.groupby(['ville', 'type'])['remboursement'].sum()

ville      type      
BORDEAUX   immobilier    15330.59
LYON       automobile      788.80
           immobilier    25275.87
MARSEILLE  automobile      839.60
           immobilier    40056.12
NICE       automobile      612.80
           immobilier    22931.44
PARIS      automobile     1426.60
           immobilier    92626.01
TOULOUSE   automobile      718.80
           immobilier    18567.38
Name: remboursement, dtype: float64

In [8]:
# Calcul de la somme et de la moyenne du remboursement par agence, 
# ainsi que le revenu maximum
prets.groupby('ville').agg(
    {'remboursement' :['sum', 'mean'],
     'revenu':'max'})

remboursement               revenu
                    sum         mean     max
ville                                       
BORDEAUX       15330.59   958.161875  3691.0
LYON           26064.67   930.881071  4771.0
MARSEILLE      40895.72   545.276267  2962.0
NICE           23544.24  1121.154286  5987.0
PARIS          94052.61  1221.462468  6807.0
TOULOUSE       19286.18   714.302963  3993.0

##### <font color = 'green'>Agréger des lignes et des colonnes (tableaux à double entrée)</font>

Nous souhaitons donc avoir la même agrégation, mais avec cette fois-ci, en lignes nos agences, et en colonnes les différents types de prêts.<br>

La méthode qui vous permettra de faire cela est la méthode .pivot_table . Celle-ci prend 4 arguments en paramètres :<br>

-> index : variable(s) placée(s) en ligne ;<br>

-> columns : variable(s) placée en colonne(s) ;<br>

-> values : variable sur laquelle on va appliquer la fonction d’agrégation ;<br>

-> aggfunc : fonction d’agrégation.<br>

La méthode permettant la transformation inverse s’appelle melt



Recours à un tableau à une entrée : méthode groupby()

In [40]:
# CAHT réalisé par chacune des agences en fonction du type de prêt contracté : 
# présentation sous la forme d'un tableau à une entrée
data1 = prets.groupby(['ville', 'type'])['remboursement'].sum().head()
data1

ville      type      
BORDEAUX   immobilier    15330.59
LYON       automobile      788.80
           immobilier    25275.87
MARSEILLE  automobile      839.60
           immobilier    40056.12
Name: remboursement, dtype: float64

In [48]:
# Suppression des champs 'ville' et 'type' en tant qu'index
data1 = data1.reset_index() 
#data1.reset_index(inplace=True) # ne fonctionnera pas car c'est une Series
data1

,index,ville,type,remboursement
0,0,BORDEAUX,immobilier,15330.59
1,1,LYON,automobile,788.80
2,2,LYON,immobilier,25275.87
3,3,MARSEILLE,automobile,839.60
4,4,MARSEILLE,immobilier,40056.12


Recours à un tableau à double entrée : méthode pivot_table()

In [49]:
# CAHT réalisé par chacune des agences en fonction du type de prêt contracté : 
# présentation sous la forme d'un tableau à double entrée
data2 = prets.pivot_table(
    index='ville', # variable où l'on va agréger les données
    columns='type', # variable(s) placée(s) en colonne
    values='remboursement', # variable sur laquelle on va appliquer la fonction
    aggfunc='sum' # fonction d'agrégation
)
data2


type,automobile,immobilier
ville,,
BORDEAUX,NaN,15330.59
LYON,788.8,25275.87
MARSEILLE,839.6,40056.12
NICE,612.8,22931.44
PARIS,1426.6,92626.01
TOULOUSE,718.8,18567.38


In [50]:
# Suppression du champ type en tant qu'index
data2.reset_index(inplace=True)
data2

type,ville,automobile,immobilier
0,BORDEAUX,NaN,15330.59
1,LYON,788.8,25275.87
2,MARSEILLE,839.6,40056.12
3,NICE,612.8,22931.44
4,PARIS,1426.6,92626.01
5,TOULOUSE,718.8,18567.38


Passage d'un tableau à double entrée à un tableau à une entrée : 
méthode melt() <br>
L'intérêt de cette méthode c'est que chaque ligne du tableau n'est pas vide

In [51]:
data2.melt(
    id_vars = 'ville', # variable pour laquelle on va agréger les données
    value_vars= ['automobile', 'immobilier'] # ensemble des colonnes qu'on souhaite passer en une seule colonne
)

,ville,type,value
0,BORDEAUX,automobile,NaN
1,LYON,automobile,788.80
2,MARSEILLE,automobile,839.60
3,NICE,automobile,612.80
4,PARIS,automobile,1426.60
5,TOULOUSE,automobile,718.80
6,BORDEAUX,immobilier,15330.59
7,LYON,immobilier,25275.87
8,MARSEILLE,immobilier,40056.12
9,NICE,immobilier,22931.44


##### <font color = 'green'>Cas pratique</font>

Suite aux résultats que vous avez fourni précédemment, le responsable du service prêt revient vous voir avec une remarque : vous l’avez probablement remarqué, mais certains clients ont contracté plusieurs prêts au sein de notre établissement. <br>Cela fausse donc potentiellement les calculs réalisés précédemment. <br>

Il formule donc des demandes additionnelles, auxquelles vous allez devoir répondre en appliquant diverses agrégations.

In [52]:
# traitement réalisés précédemment
prets = pd.read_csv('https://raw.githubusercontent.com/benjaminmrl/data-4452741/main/prets.csv')

# calcul du taux d'endettement
prets['taux_endettement'] = round(prets['remboursement'] * 100 / prets['revenu'], 2)

# renommer taux en taux_interet
prets.rename(columns={'taux':'taux_interet'}, inplace=True)

# calculer le cout total du pret
prets['cout_total'] = prets['remboursement'] * prets['duree']

# calculer les bénéfices mensuels réalisés
prets['benefices'] = round((prets['cout_total'] * prets['taux_interet']/100)/(24), 2)

# création d'une variable risque
prets['risque'] = 'Non'
prets.loc[prets['taux_endettement'] > 35, 'risque'] = 'Oui'

prets.head()

,identifiant,ville,CP,revenu,remboursement,duree,type,taux_interet,taux_endettement,cout_total,benefices,risque
0,0,TOULOUSE,31100,3669.0,1130.05,240,immobilier,1.168,30.80,271212.0,131.99,Non
1,1,PARIS,75009,5310.0,240.00,64,automobile,3.701,4.52,15360.0,23.69,Non
2,1,PARIS,75009,5310.0,1247.85,300,immobilier,1.173,23.50,374355.0,182.97,Non
3,2,MARSEILLE,13010,1873.0,552.54,240,immobilier,0.972,29.50,132609.6,53.71,Non
4,3,MARSEILLE,13010,1684.0,586.03,180,immobilier,1.014,34.80,105485.4,44.57,Non


Pourriez vous créez un dataframe profil client pour pallier ce problème ? <br>

Par profil client, nous entendons un dataframe où il y n'y a qu'une seule ligne par client, avec le résumé de ses informations (somme remboursement, du taux d'endettement, du cout_total et des bénéfices réalisés)

In [53]:
profil_clients = (prets.groupby('identifiant')[[
    'remboursement', 'taux_endettement', 'cout_total', 'benefices']]
 .sum().reset_index())
profil_clients.head()

,identifiant,remboursement,taux_endettement,cout_total,benefices
0,0,1130.05,30.80,271212.0,131.99
1,1,1487.85,28.02,389715.0,206.66
2,2,552.54,29.50,132609.6,53.71
3,3,586.03,34.80,105485.4,44.57
4,4,423.61,28.70,101666.4,51.21


Recalculez le nombre exact de personnes en situation bancaire risquée à partir du taux d'endettemment :

In [60]:
nombre_client_risque = prets.loc[prets['taux_endettement'] >= 35].shape[0]
print(f'Il y a {nombre_client_risque} clients qui ont dépassé le seuil autorisé')

Il y a 60 clients qui ont dépassé le seuil autorisé


Calculez le bénéfice dégagé par chacune de nos agences, par types de prêts. Vous présenterez vos résultats sous la forme d'un tableau simple (via un group_by) :

In [65]:
prets.groupby(['ville', 'type'])['benefices'].sum()


ville      type      
BORDEAUX   immobilier    1681.32
LYON       automobile      47.39
           immobilier    2967.41
MARSEILLE  automobile      61.36
           immobilier    4334.55
NICE       automobile      31.15
           immobilier    2651.60
PARIS      automobile      99.50
           immobilier    9916.85
TOULOUSE   automobile      61.22
           immobilier    2076.85
Name: benefices, dtype: float64

Pour aller plus loin, on souhaite avoir un tableau à double entrée (via un pivot_table) présentant cette fois ci les bénéfices moyen réalisés par chaque agence, pour chaque type de prêt :

In [67]:
benef_ville = prets.pivot_table(
    index='ville',
    columns='type',
    values='benefices',
    aggfunc='mean')
benef_ville

type,automobile,immobilier
ville,,
BORDEAUX,NaN,105.082500
LYON,15.796667,118.696400
MARSEILLE,20.453333,60.202083
NICE,15.575000,139.557895
PARIS,19.900000,137.734028
TOULOUSE,20.406667,86.535417


Quelle ville semble la plus intéressante où développer les prêts immobiliers ?

In [70]:
benef_ville.sort_values(by='immobilier', ascending=False)

type,automobile,immobilier
ville,,
NICE,15.575000,139.557895
PARIS,19.900000,137.734028
LYON,15.796667,118.696400
BORDEAUX,NaN,105.082500
TOULOUSE,20.406667,86.535417
MARSEILLE,20.453333,60.202083
